In [1]:
from ultralytics import YOLO
import os
import geopandas as gpd
from shapely.geometry import Polygon

C:\Users\phili\AppData\Local\Temp\ipykernel_106428\3450588983.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
model = YOLO("../runs/detect/train/weights/best.pt")  # pretrained YOLOv8n model

In [22]:
image_file = "../data/tilesZugeschnittenUndJsonZurGeoRef/tile_1536_11264.tif"

# tile_1536_11264
# tile_1024_10240

In [23]:
results = model(image_file)
print(results[0].boxes)


image 1/1 C:\Users\phili\Documents\Studium\Master_Geographie\5_Semester\S_Deep_Learning\beet-segmentation\src\..\data\tilesZugeschnittenUndJsonZurGeoRef\tile_1536_11264.tif: 512x512 2 sugar beets, 47.7ms
Speed: 2.0ms preprocess, 47.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0.])
conf: tensor([0.5101, 0.3447])
data: tensor([[2.4349e+02, 1.4902e+02, 3.7506e+02, 2.4648e+02, 5.1014e-01, 0.0000e+00],
        [2.2742e+02, 1.4652e+02, 3.6526e+02, 2.2329e+02, 3.4469e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (512, 512)
shape: torch.Size([2, 6])
xywh: tensor([[309.2744, 197.7469, 131.5725,  97.4607],
        [296.3383, 184.9025, 137.8449,  76.7694]])
xywhn: tensor([[0.6041, 0.3862, 0.2570, 0.1904],
        [0.5788, 0.3611, 0.2692, 0.1499]])
xyxy: tensor([[243.4881, 149.0165, 375.0607, 246.4772],
        [227.4158, 146.5178, 365.2607, 223.2872]])
xyxyn: tensor([[0.4756, 0.2910, 0.73

In [24]:
for result in results:
    boxes = result.boxes
    if boxes.shape[0] != 0:
        pixel_xmin = boxes.xyxy[0][0].item()
        pixel_ymin = boxes.xyxy[0][1].item()
        pixel_xmax = boxes.xyxy[0][2].item()
        pixel_ymax = boxes.xyxy[0][3].item()
    else:
        continue

In [25]:
print(pixel_xmin, pixel_ymin, pixel_xmax, pixel_ymax)

243.48814392089844 149.0165252685547 375.0606689453125 246.4772491455078


In [26]:
from osgeo import gdal
import numpy as np

def pixel_to_geo(pixel_x, pixel_y, geotransform):
    geo_x = geotransform[0] + pixel_x * geotransform[1] + pixel_y * geotransform[2]
    geo_y = geotransform[3] + pixel_x * geotransform[4] + pixel_y * geotransform[5]
    return geo_x, geo_y

In [27]:
# Example usage
dataset = gdal.Open(image_file)
if dataset is None:
    raise FileNotFoundError("Image file not found")

# Get geotransform
geotransform = dataset.GetGeoTransform()

In [28]:
print(geotransform)

(576496.6226073955, 0.00040246724450860487, 0.0, 5516317.77482754, 0.0, -0.00040246724450860487)


In [29]:
# Convert pixel coordinates to geographic coordinates
geo_xmin, geo_ymin = pixel_to_geo(pixel_xmin, pixel_ymin, geotransform)
geo_xmax, geo_ymax = pixel_to_geo(pixel_xmax, pixel_ymax, geotransform)
print("Geographic coordinates:", geo_xmin, geo_ymin, geo_xmax, geo_ymax)

# Close the dataset
dataset = None

Geographic coordinates: 576496.7206033979 5516317.71485327 576496.7735570294 5516317.675628521


In [30]:
# Create a bounding box as a Shapely Polygon object
bounding_box = Polygon([(geo_xmin, geo_ymin), (geo_xmax, geo_ymin), (geo_xmax, geo_ymax), (geo_xmin, geo_ymax)])

# Print the bounding box
print("Bounding box as a Shapely Polygon object:", bounding_box)

Bounding box as a Shapely Polygon object: POLYGON ((576496.7206033979 5516317.71485327, 576496.7735570294 5516317.71485327, 576496.7735570294 5516317.675628521, 576496.7206033979 5516317.675628521, 576496.7206033979 5516317.71485327))


In [33]:
# Create a GeoDataFrame with the bounding box
gdf = gpd.GeoDataFrame(geometry=[bounding_box])

# Buffer single polygons
buffered_gdf = gdf.buffer(0.025)

buffered_gdf.type

0    Polygon
dtype: object

In [32]:
# Define the output file path
output_file = "bounding_box4.geojson"

# Export the GeoDataFrame to a GeoJSON file
buffered_gdf.to_file(output_file, driver='GeoJSON')

print("Bounding box exported to:", output_file)

Bounding box exported to: bounding_box4.geojson
